# HW06 : POS Tagging & Named Entity Recognition

ในหัวข้อนี้จะได้เรียนรู้การสร้างโมเดลสำหรับทำนาย POS และ NE โดยใช้วิธีต่างๆ ดังนี้

1) Hidden Markov Model (HMM)

2) Context Random Field (CR)



# HMM POS Tagging

ในส่วนนี้เราจะมาลองทำ POS Tagger โดยใช้ HMM

ลองกับภาษาอังกฤษ และดัดแปลงเป็นภาษาไทย โดยอาศัยโค้ดจาก
https://gist.github.com/girish1511/9e3b6bc2a4a2eca0b904aff7e7fe7685



In [1]:
# Importing the required libraries
import nltk
import numpy as np
import itertools

In [2]:
nltk.download('brown')
from nltk.corpus import brown

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


In [3]:
#Getting the tagged sentences
sent_tag = brown.tagged_sents()
mod_sent_tag=[]
for s in sent_tag:
  s.insert(0,('##','##'))
  s.append(('&&','&&'))
  mod_sent_tag.append(s)

In [ ]:
#[[('##', '##'),('The', 'AT'),('Fulton', 'NP-TL'),('&&', '&&')],[]]
mod_sent_tag=[]
fname = '/content/LST20_Corpus/train_clean.txt'
with open(fname) as fp:
    lines = fp.readlines()
    tmp_sen = []
    for line in lines:

        if line.strip()=="":
            if len(tmp_sen) > 0:
                tmp_sen.insert(0,('##','##'))
                tmp_sen.append(('&&','&&'))
                mod_sent_tag.append(tmp_sen)

                tmp_sen = []

            continue

        _x = line.split('\t')

        tmp_sen.append((_x[0],_x[1]))



In [ ]:
mod_sent_tag[:2]

In [ ]:
#Splitting the data for train and test
split_num = int(len(mod_sent_tag)*0.9)
train_data = mod_sent_tag[0:split_num]
test_data = mod_sent_tag[split_num:]

In [ ]:
#Creating a dictionary whose keys are tags and values contain words which were assigned the correspoding tag
# ex:- 'TAG':{word1: count(word1,'TAG')}
train_word_tag = {}
for s in train_data:
  for (w,t) in s:
    w=w.lower()
    try:
      try:
        train_word_tag[t][w]+=1
      except:
        train_word_tag[t][w]=1
    except:
      train_word_tag[t]={w:1}

In [ ]:
#Calculating the emission probabilities using train_word_tag
train_emission_prob={}
for k in train_word_tag.keys():
  train_emission_prob[k]={}
  count = sum(train_word_tag[k].values())
  for k2 in train_word_tag[k].keys():
    train_emission_prob[k][k2]=train_word_tag[k][k2]/count

In [ ]:
#Estimating the bigram of tags to be used for transition probability
bigram_tag_data = {}
for s in train_data:
  bi=list(nltk.bigrams(s))
  for b1,b2 in bi:
    try:
      try:
        bigram_tag_data[b1[1]][b2[1]]+=1
      except:
        bigram_tag_data[b1[1]][b2[1]]=1
    except:
      bigram_tag_data[b1[1]]={b2[1]:1}

In [ ]:
#Calculating the probabilities of tag bigrams for transition probability
bigram_tag_prob={}
for k in bigram_tag_data.keys():
  bigram_tag_prob[k]={}
  count=sum(bigram_tag_data[k].values())
  for k2 in bigram_tag_data[k].keys():
    bigram_tag_prob[k][k2]=bigram_tag_data[k][k2]/count

In [ ]:
#Calculating the possible tags for each word
#Note: Here we have used the whole data(Train+Test)
#Reason: There may be some words which are not present in train data but are present in test data
tags_of_tokens = {}
count=0
for s in train_data:
  for (w,t) in s:
    w=w.lower()
    try:
      if t not in tags_of_tokens[w]:
        tags_of_tokens[w].append(t)
    except:
      l = []
      l.append(t)
      tags_of_tokens[w] = l

for s in test_data:
  for (w,t) in s:
    w=w.lower()
    try:
      if t not in tags_of_tokens[w]:
        tags_of_tokens[w].append(t)
    except:
      l = []
      l.append(t)
      tags_of_tokens[w] = l

In [ ]:
#Dividing the test data into test words and test tags
test_words=[]
test_tags=[]
for s in test_data:
  temp_word=[]
  temp_tag=[]
  for (w,t) in s:
    temp_word.append(w.lower())
    temp_tag.append(t)
  test_words.append(temp_word)
  test_tags.append(temp_tag)


In [ ]:
#Executing the Viterbi Algorithm
predicted_tags = []
#test_words = test_words[0:1]
print(test_words)
#intializing the predicted tags
for x in range(len(test_words)):   # for each tokenized sentence in the test data
  s = test_words[x]
  #storing_values is a dictionary which stores the required values
  #ex: storing_values = {step_no.:{state1:[previous_best_state,value_of_the_state]}}
  storing_values = {}
  for q in range(len(s)):
    step = s[q]
    #for the starting word of the sentence
    if q == 1:
      storing_values[q] = {}
      tags = tags_of_tokens[step]
      for t in tags:
        #this is applied since we do not know whether the word in the test data is present in train data or not
        try:
          storing_values[q][t] = ['##',bigram_tag_prob['##'][t]*train_emission_prob[t][step]]
        #if word is not present in the train data but present in test data we assign a very low probability of 0.0001
        except:
          storing_values[q][t] = ['##',0.0001]#*train_emission_prob[t][step]]

    #if the word is not at the start of the sentence
    if q>1:
      storing_values[q] = {}
      previous_states = list(storing_values[q-1].keys())   # loading the previous states
      current_states  = tags_of_tokens[step]               # loading the current states
      #calculation of the best previous state for each current state and then storing
      #it in storing_values
      #print('previous_states',q,previous_states)
      for t in current_states:
        temp = []
        for pt in previous_states:
          try:
            temp.append(storing_values[q-1][pt][1]*bigram_tag_prob[pt][t]*train_emission_prob[t][step])
          except:
            temp.append(storing_values[q-1][pt][1]*0.0001)
        #หา max prob
        max_temp_index = temp.index(max(temp))
        best_pt = previous_states[max_temp_index]
        #update best
        storing_values[q][t]=[best_pt,max(temp)]

        #print(q,t,max_temp_index,best_pt,max(temp),storing_values[q][t])


  #Backtracing to extract the best possible tags for the sentence
  pred_tags = []
  total_steps_num = storing_values.keys()
  last_step_num = max(total_steps_num)
  #print(storing_values)
  #print(total_steps_num,last_step_num)
  for bs in range(len(total_steps_num)):
    step_num = last_step_num - bs
    if step_num == last_step_num:
      pred_tags.append('&&')
      pred_tags.append(storing_values[step_num]['&&'][0])
    else:
      #print(storing_values[step_num][pred_tags[len(pred_tags)-1]],pred_tags[len(pred_tags)-1])
      pred_tags.append(storing_values[step_num][pred_tags[len(pred_tags)-1]][0])
  predicted_tags.append(list(reversed(pred_tags)))
  #print(predicted_tags)

[]


In [ ]:
#Calculating the accuracy based on tagging each word in the test data.
right = 0
wrong = 0
for i in range(len(test_tags)):
  gt = test_tags[i]
  pred = predicted_tags[i]
  for h in range(len(gt)):
    if gt[h] == pred[h]:
      right = right+1
    else:
      wrong = wrong +1

print('Accuracy on the test data is: ',right/(right+wrong))
print('Loss on the test data is: ',wrong/(right+wrong))

Accuracy on the test data is:  0.9473042786182785
Loss on the test data is:  0.05269572138172154


# Thai HMM-based POS Tagging

จากโค้ดข้างตอนให้นักศึกษาปรับโค้ดสร้างโมเดล HMM สำหรับข้อมูลภาษาไทย

โดยคลังข้อมูลที่ใช้จะใช้ข้อมูล LST20

ดาวน์โหลดได้ที่ https://aiforthai.in.th/download.php?c=LST20

โดยสามารถดาวน์โหลดได้ตามขึ้นตอนดังนี้



In [4]:
!pip install -U gdown

  Attempting uninstall: gdown
    Found existing installation: gdown 4.7.3
    Uninstalling gdown-4.7.3:
      Successfully uninstalled gdown-4.7.3


In [5]:
#https://drive.google.com/file/d/1RQYB_ZGiCEMHNRlTN4vQgXK-9UifBmJQ/view?usp=drive_link

!gdown 1RQYB_ZGiCEMHNRlTN4vQgXK-9UifBmJQ

Downloading...
From: https://drive.google.com/uc?id=1RQYB_ZGiCEMHNRlTN4vQgXK-9UifBmJQ
To: /content/AIFORTHAI-LST20Corpus.tar.gz
100% 13.6M/13.6M [00:00<00:00, 95.2MB/s]


In [6]:
!tar -xvf AIFORTHAI-LST20Corpus.tar.gz

เอาต์พุตของการสตรีมมีการตัดเหลือเพียง 5000 บรรทัดสุดท้าย
LST20_Corpus/train/._T12371.txt
LST20_Corpus/train/T12371.txt
LST20_Corpus/train/._T11678.txt
LST20_Corpus/train/T11678.txt
LST20_Corpus/train/._T10566.txt
LST20_Corpus/train/T10566.txt
LST20_Corpus/train/._T00775.txt
LST20_Corpus/train/T00775.txt
LST20_Corpus/train/._T12417.txt
LST20_Corpus/train/T12417.txt
LST20_Corpus/train/._T12403.txt
LST20_Corpus/train/T12403.txt
LST20_Corpus/train/._T10214.txt
LST20_Corpus/train/T10214.txt
LST20_Corpus/train/._T01319.txt
LST20_Corpus/train/T01319.txt
LST20_Corpus/train/._T00761.txt
LST20_Corpus/train/T00761.txt
LST20_Corpus/train/._T01457.txt
LST20_Corpus/train/T01457.txt
LST20_Corpus/train/._T00991.txt
LST20_Corpus/train/T00991.txt
LST20_Corpus/train/._T11644.txt
LST20_Corpus/train/T11644.txt
LST20_Corpus/train/._T00749.txt
LST20_Corpus/train/T00749.txt
LST20_Corpus/train/._T13053.txt
LST20_Corpus/train/T13053.txt
LST20_Corpus/train/._T11122.txt
LST20_Corpus/train/T11122.txt
LST20_Corpus/

In [7]:
!head -n 20 LST20_Corpus/eval/T01608.txt

สลด	VV	O	B_CLS
ผัว	NN	O	I_CLS
เมีย	NN	O	I_CLS
เฒ่า	NN	O	I_CLS
ถูก	AX	O	I_CLS
ไฟ	NN	O	I_CLS
ช็อต	VV	O	I_CLS
กอด	VV	O	I_CLS
กัน	AV	O	I_CLS
ตาย	VV	O	I_CLS
_	PU	O	I_CLS
สังเวย	VV	O	I_CLS
ฝน	NN	O	I_CLS
ตก	VV	O	I_CLS
-	PU	O	I_CLS
น้ำ	NN	O	I_CLS
ท่วม	VV	O	E_CLS

สอง	NU	O	B_CLS
สามี	NN	O	I_CLS


In [8]:
def clean_data(fname):
    with open(fname.replace(".txt","_clean.txt"),"w") as fw:
        with open(fname) as fp:
            lines = fp.readlines()

            for line in lines:
                #print(line)

                if line.strip()=="":
                    fw.write("\n")
                else:
                    #print(line.split("\t"))
                    if len(line.split("\t"))==4:
                        fw.write(line)

In [9]:
#create train/test/val data
!cat LST20_Corpus/train/T*.txt > LST20_Corpus/train.txt
!cat LST20_Corpus/eval/T*.txt > LST20_Corpus/val.txt
!cat LST20_Corpus/test/T*.txt > LST20_Corpus/test.txt


In [10]:
clean_data("LST20_Corpus/train.txt")
clean_data("LST20_Corpus/val.txt")
clean_data("LST20_Corpus/test.txt")

In [11]:
!head -n 10 LST20_Corpus/train_clean.txt

สุรยุทธ์	NN	B_PER	B_CLS
ยัน	VV	O	I_CLS
ปฏิเสธ	VV	O	I_CLS
ลงนาม	VV	O	I_CLS
_	PU	O	I_CLS
MOU	NN	O	I_CLS
_	PU	O	I_CLS
กับ	PS	O	I_CLS
อียู	NN	B_ORG	I_CLS
ไม่	NG	O	I_CLS


In [ ]:
#YOUR CODE HERE


# CRF for NER

ในส่วนนี้เราจะสร้างแบบจำลอง NER สำหรับภาษาไทยโดยใช้คลังข้อมูล LST20 กัน

โดยจะอาศัยตัวอย่างโค้ดจาก

https://albertauyeung.github.io/2017/06/17/python-sequence-labelling-with-crf.html/

In [12]:
!pip install python-crfsuite

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.8 MB/s eta 0:00:00


## เตรียมข้อมูล

โหลดข้อมูลให้ตรงตามรูปแบบ โดยแต่ละประโยคจะมีหน้าตา ดังนี้ [(w, pos, label),(w, pos, label),...]

In [ ]:
#YOU CODE HERE
#โหลดข้อมูลเก็บที่ตัวแปร docs

In [ ]:
import nltk
data = []
for i, doc in enumerate(docs):

    # Obtain the list of tokens in the document
    #tokens = [t for t, label in doc]

    # Perform POS tagging
    #tagged = nltk.pos_tag(tokens)

    # Take the word, POS tag, and its label
    #data.append([(w, pos, label) for (w, label), (word, pos) in zip(doc, tagged)])

    #เพิ่มโค้ดเพื่อให้โหลดข้อมูลหน้าตาประมาณนี้
    #data.append([(w, pos, label),(w, pos, label),...])
    continue


In [14]:
#[[('##', '##'),('The', 'AT'),('Fulton', 'NP-TL'),('&&', '&&')],[]]
data=[]
fname = '/content/LST20_Corpus/train_clean.txt'
with open(fname) as fp:
    lines = fp.readlines()
    tmp_sen = []
    for line in lines:

        if line.strip()=="":
            if len(tmp_sen) > 0:
                #tmp_sen.insert(0,('##','##'))
                #tmp_sen.append(('&&','&&'))
                data.append(tmp_sen)
                tmp_sen = []

            continue

        _x = line.split('\t')

        tmp_sen.append((_x[0],_x[1],_x[2]))



In [15]:
data[:2]

[[('สุรยุทธ์', 'NN', 'B_PER'),
  ('ยัน', 'VV', 'O'),
  ('ปฏิเสธ', 'VV', 'O'),
  ('ลงนาม', 'VV', 'O'),
  ('_', 'PU', 'O'),
  ('MOU', 'NN', 'O'),
  ('_', 'PU', 'O'),
  ('กับ', 'PS', 'O'),
  ('อียู', 'NN', 'B_ORG'),
  ('ไม่', 'NG', 'O'),
  ('กระทบ', 'VV', 'O'),
  ('สัมพันธ์', 'NN', 'O')],
 [('1', 'NU', 'B_DTM'),
  ('_', 'PU', 'I_DTM'),
  ('กันยายน', 'NN', 'I_DTM'),
  ('_', 'PU', 'I_DTM'),
  ('2550', 'NU', 'E_DTM'),
  ('_', 'PU', 'O'),
  ('12:21', 'NU', 'B_DTM'),
  ('_', 'PU', 'I_DTM'),
  ('น.', 'CL', 'E_DTM')]]

## สร้าง Feature

โดยเราจะใช้ Feature ประมาณนี้


*   The word w itself (converted to lowercase for normalisation)
*   The prefix/suffix of w (e.g. -ion)
*   The words surrounding w, such as the previous and the next word
*   Whether w is in uppercase or lowercase
*   Whether w is a number, or contains digits
*   The POS tag of w, and those of the surrounding words
*   Whether w is or contains a special character (e.g. hypen, dollar sign)


In [16]:
def word2features(doc, i):
    word = doc[i][0]
    postag = doc[i][1]

    # Common features for all words
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag
    ]
    #print(features)

    # Features for words that are not
    # at the beginning of a document
    if i > 0:
        word1 = doc[i-1][0]
        postag1 = doc[i-1][1]
        #prev_word
        features.extend([
            'prev_word_lower='+ word.lower(),
            'prev_word.isdigit=%s' % word.isdigit()
        ])
    else:
        # Indicate that it is the 'beginning of a document'
        features.append('BOS')

    # Features for words that are not
    # at the end of a document
    if i < len(doc)-1:
        word1 = doc[i+1][0]
        postag1 = doc[i+1][1]

        features.extend([])
    else:
        # Indicate that it is the 'end of a document'
        features.append('EOS')

    return features

In [17]:
word2features(data[0],2)

['bias',
 'word.lower=ปฏิเสธ',
 'word[-3:]=เสธ',
 'word[-2:]=สธ',
 'word.isupper=False',
 'word.istitle=False',
 'word.isdigit=False',
 'postag=VV',
 'prev_word_lower=ปฏิเสธ',
 'prev_word.isdigit=False']

In [18]:
from sklearn.model_selection import train_test_split

# A function for extracting features in documents
def extract_features(doc):
    return [word2features(doc, i) for i in range(len(doc))]

# A function fo generating the list of labels for each document
def get_labels(doc):
    return [label for (token, postag, label) in doc]

X = [extract_features(doc) for doc in data]
y = [get_labels(doc) for doc in data]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [19]:
X[0]

[['bias',
  'word.lower=สุรยุทธ์',
  'word[-3:]=ทธ์',
  'word[-2:]=ธ์',
  'word.isupper=False',
  'word.istitle=False',
  'word.isdigit=False',
  'postag=NN',
  'BOS'],
 ['bias',
  'word.lower=ยัน',
  'word[-3:]=ยัน',
  'word[-2:]=ัน',
  'word.isupper=False',
  'word.istitle=False',
  'word.isdigit=False',
  'postag=VV',
  'prev_word_lower=ยัน',
  'prev_word.isdigit=False'],
 ['bias',
  'word.lower=ปฏิเสธ',
  'word[-3:]=เสธ',
  'word[-2:]=สธ',
  'word.isupper=False',
  'word.istitle=False',
  'word.isdigit=False',
  'postag=VV',
  'prev_word_lower=ปฏิเสธ',
  'prev_word.isdigit=False'],
 ['bias',
  'word.lower=ลงนาม',
  'word[-3:]=นาม',
  'word[-2:]=าม',
  'word.isupper=False',
  'word.istitle=False',
  'word.isdigit=False',
  'postag=VV',
  'prev_word_lower=ลงนาม',
  'prev_word.isdigit=False'],
 ['bias',
  'word.lower=_',
  'word[-3:]=_',
  'word[-2:]=_',
  'word.isupper=False',
  'word.istitle=False',
  'word.isdigit=False',
  'postag=PU',
  'prev_word_lower=_',
  'prev_word.isdigit=F

In [20]:
import pycrfsuite
trainer = pycrfsuite.Trainer(verbose=True)

# Submit training data to the trainer
for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

# Set the parameters of the model
trainer.set_params({
    # coefficient for L1 penalty
    'c1': 0.1,

    # coefficient for L2 penalty
    'c2': 0.01,

    # maximum number of iterations
    'max_iterations': 5,

    # whether to include transitions that
    # are possible, but not observed
    'feature.possible_transitions': True
})

# Provide a file name as a parameter to the train function, such that
# the model will be saved to the file when training is finished
trainer.train('crf1.model')

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 153315
Seconds required: 3.667

L-BFGS optimization
c1: 0.100000
c2: 0.010000
num_memories: 6
max_iterations: 5
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 3841113.921033
Feature norm: 1.000000
Error norm: 3451855.784957
Active features: 152571
Line search trials: 1
Line search step: 0.000000
Seconds required for this iteration: 19.497

***** Iteration #2 *****
Loss: 2897215.440825
Feature norm: 3.085397
Error norm: 967424.266571
Active features: 149273
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 19.209

***** Iteration #3 *****
Loss: 2487069.778364
Feature norm: 2.659027
Error norm: 887390.222731
Active features: 146094
Line search trials: 1
Line search step: 1.000000
Second

## ทดสอบและวัดประสิทธิภาพ

In [21]:
tagger = pycrfsuite.Tagger()
tagger.open('crf1.model')


In [22]:
tagger.info()

In [23]:
tagg = []
for y in y_train:
  tagg += y

tag_set = list(set(tagg))

In [24]:
tag_set

['I_DTM',
 'B_NAME',
 'E_DES',
 'I_DES',
 'B_TRM',
 'E_BRN',
 'E_DTM',
 'I',
 'B_LOC',
 'I_NUM',
 'E_TTL',
 'I_BRN',
 'I_LOC',
 '__',
 'B_BRN',
 'E_TRM',
 'O',
 'I_ORG',
 'MEA_BI',
 'I_PER',
 'I_TTL',
 'B_TTL',
 'PER_I',
 'E_ORG',
 'B_ORG',
 'B_DTM',
 'B_NUM',
 'E_PER',
 'B_DES',
 'B_PER',
 'E_NUM',
 'E_LOC',
 'B_MEA',
 'B',
 'E_MEA',
 'DDEM',
 'I_TRM',
 'I_MEA']

In [ ]:
import numpy as np
from sklearn.metrics import classification_report

# Convert the sequences of tags into a 1-dimensional array
_predictions = [tag for row in y_pred for tag in row]
_truths = [tag for row in y_test for tag in row]


focus_tag = ['B_LOC', 'B_DES', 'I_MEA', 'B_ORG', 'E_ORG', 'E_BRN', 'B_BRN', 'I_TRM', 'E_TTL', 'B_TTL','B_MEA',
 'I_LOC',
 'E_DES',
 'I_BRN',
 'I_TTL',
 'I_NUM',
 'I_ORG',
 'I_PER',
 'E_TRM',
 'B_PER',
 'B_TRM',
 'E_DTM',
 'E_PER',
 'B_NUM',
 'E_LOC',
 'I_DTM',
 'E_NUM',
 'B_NAME',
 'B_DTM',
 'E_MEA',
 'I_DES']

In [ ]:
truths = []
predictions = []
for x,y in zip(_truths,_predictions):
  if x in focus_tag:
    truths.append(x)
    predictions.append(y)

In [ ]:
print(predictions[:50])
print(truths[:50])

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['B_BRN', 'B_MEA', 'B_ORG', 'B_ORG', 'B_NUM', 'B_NUM', 'E_MEA', 'B_NUM', 'B_NUM', 'I_NUM', 'I_NUM', 'E_NUM', 'B_MEA', 'B_ORG', 'B_ORG', 'E_ORG', 'B_ORG', 'B_ORG', 'B_DES', 'B_NUM', 'B_ORG', 'B_MEA', 'I_MEA', 'I_MEA', 'E_MEA', 'B_LOC', 'B_LOC', 'B_TTL', 'B_PER', 'I_PER', 'E_PER', 'B_PER', 'B_PER', 'B_MEA', 'B_MEA', 'B_ORG', 'B_LOC', 'E_LOC', 'B_NUM', 'B_MEA', 'B_MEA', 'B_MEA', 'B_PER', 'I_PER', 'E_PER', 'B_ORG', 'B_PER', 'I_PER', 'E_PER', 'B_DES']


In [ ]:
# Print out the classification report
print(classification_report(
    truths, predictions))

# Assignment

1) ให้ใช้ข้อมูล LST20 มาสร้างเป็นโมเดลสำหรับทำนาย NE ของประโยคภาษาไทยโดยใช้ CRF
โดดให้สร้างโมเดล 3 โมเดล

**Model 1** : ใช้ Feature เฉพาะคำปัจจุบัน

**Model 2** : ใช้ Feature ของคำปัจจุบัน คำก่อนหน้าและคำถัดไป (w_i-1,w_i,w_i+1)

**Model 3** : ใช้ Fature เหมือนโมเดล 2 แต่เพิ่มคำรอบข้างอีก 1 คำ (w_i-2,w_i-1,w_i,w_i+1,w_i+2)

2) ให้ทำการวัดผลแสดงค่า Precision Recall F1-score


In [ ]:
#YOUR CODE HERE

# model 1

---



In [26]:
def word2features(doc, i):
    word = doc[i][0]
    postag = doc[i][1]

    # Common features for all words
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag
    ]
    print(features)

    # Features for words that are not
    # at the beginning of a document
    if i > 0:
        word1 = doc[i-1][0]
        postag1 = doc[i-1][1]
        #prev_word
        features.extend([
            # 'prev_word_lower='+ word1.lower(),
            # 'prev_word.isdigit=%s' % word1.isdigit()
        ])
    else:
        # Indicate that it is the 'beginning of a document'
        features.append('BOS')

    # Features for words that are not
    # at the end of a document
    if i < len(doc)-1:
        word1 = doc[i+1][0]
        postag1 = doc[i+1][1]

        features.extend([
            # 'next_word_lower='+ word1.lower(),
            # 'next_word.isdigit=%s' % word1.isdigit()
        ])
    else:
        # Indicate that it is the 'end of a document'
        features.append('EOS')

    return features

In [27]:
import pycrfsuite
trainer = pycrfsuite.Trainer(verbose=True)

# Submit training data to the trainer
for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

# Set the parameters of the model
trainer.set_params({
    # coefficient for L1 penalty
    'c1': 0.1,

    # coefficient for L2 penalty
    'c2': 0.01,

    # maximum number of iterations
    'max_iterations': 50,

    # whether to include transitions that
    # are possible, but not observed
    'feature.possible_transitions': True
})

# Provide a file name as a parameter to the train function, such that
# the model will be saved to the file when training is finished
trainer.train('crf1.model')

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 153315
Seconds required: 3.631

L-BFGS optimization
c1: 0.100000
c2: 0.010000
num_memories: 6
max_iterations: 50
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 3841113.921033
Feature norm: 1.000000
Error norm: 3451855.784957
Active features: 152571
Line search trials: 1
Line search step: 0.000000
Seconds required for this iteration: 19.850

***** Iteration #2 *****
Loss: 2897215.440825
Feature norm: 3.085397
Error norm: 967424.266571
Active features: 149273
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 19.800

***** Iteration #3 *****
Loss: 2487069.778364
Feature norm: 2.659027
Error norm: 887390.222731
Active features: 146094
Line search trials: 1
Line search step: 1.000000
Secon

In [28]:
tagger = pycrfsuite.Tagger()
tagger.open('crf1.model')
y_pred = [tagger.tag(xseq) for xseq in X_test]

# Let's take a look at a random sample in the testing set
i = 12
for x, y in zip(y_pred[i], [x[1].split("=")[1] for x in X_test[i]]):
    print("%s (%s)" % (y, x))

'''
The following will be printed:

sci-med (N)
life (N)
systems (N)
inc (N)
said (I)
its (I)
directors (I)
approved (I)
a (I)
previously (I)
...
'''

ขณะ (O)
เดียว (O)
กัน (O)
_ (O)
หนังสือพิมพ์ (O)
_ (O)
เดลิเมล์ (O)
_ (O)
รายงาน (O)
ว่า (O)
_ (O)
ปัญหา (O)
เรื่อง (O)
สัญญา (O)
ระหว่าง (O)
มาดริด (B_ORG)
กับ (O)
_ (O)
คริสเตียโน (B_PER)
_ (I_PER)
โรนัลโด (E_PER)
_ (O)
ได้ (O)
ข้อ (O)
ยุติ (O)
แล้ว (O)
_ (O)
หลัง (O)
_ (O)
จอร์เก (B_PER)
_ (I_PER)
เมนเดส (E_PER)
_ (O)
เอเยนต์ (O)
ของ (O)
โรนัลโด (B_PER)
_ (O)
ตกลง (O)
แบ่ง (O)
ผล (O)
ประโยชน์ (O)
กับ (O)
ว่าที่ (O)
สังกัด (O)
ใหม่ (O)
ของ (O)
ปีก (O)
โปรตุเกส (B_ORG)
ราย (O)
นี้ (O)
ได้ (O)


'\nThe following will be printed:\n\nsci-med (N)\nlife (N)\nsystems (N)\ninc (N)\nsaid (I)\nits (I)\ndirectors (I)\napproved (I)\na (I)\npreviously (I)\n...\n'

In [29]:
import numpy as np
from sklearn.metrics import classification_report

# Convert the sequences of tags into a 1-dimensional array
_predictions = [tag for row in y_pred for tag in row]
_truths = [tag for row in y_test for tag in row]


focus_tag = ['B_LOC', 'B_DES', 'I_MEA', 'B_ORG', 'E_ORG', 'E_BRN', 'B_BRN', 'I_TRM', 'E_TTL', 'B_TTL','B_MEA',
 'I_LOC',
 'E_DES',
 'I_BRN',
 'I_TTL',
 'I_NUM',
 'I_ORG',
 'I_PER',
 'E_TRM',
 'B_PER',
 'B_TRM',
 'E_DTM',
 'E_PER',
 'B_NUM',
 'E_LOC',
 'I_DTM',
 'E_NUM',
 'B_NAME',
 'B_DTM',
 'E_MEA',
 'I_DES']

In [30]:
truths = []
predictions = []
for x,y in zip(_truths,_predictions):
  if x in focus_tag:
    truths.append(x)
    predictions.append(y)

In [31]:
# Print out the classification report
print(classification_report(truths, predictions))

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

              precision    recall  f1-score   support

           B       0.00      0.00      0.00         0
       B_BRN       0.00      0.00      0.00       106
       B_DES       0.97      0.75      0.85      3885
       B_DTM       0.92      0.74      0.82      3133
       B_LOC       0.84      0.67      0.75      7380
       B_MEA       0.72      0.58      0.64      6980
      B_NAME       0.00      0.00      0.00         1
       B_NUM       0.58      0.47      0.52      3058
       B_ORG       0.89      0.64      0.74     10238
       B_PER       0.96      0.83      0.89      9877
       B_TRM       0.88      0.28      0.43       215
       B_TTL       0.99      0.95      0.97      4498
       E_BRN       0.00      0.00      0.00        31
       E_DES       0.73      0.29      0.41       395
       E_DTM       0.91      0.83      0.87      2618
       E_LOC       0.77      0.74      0.76      3388
       E_MEA       0.69      0.76      0.72      2172
       E_NUM       0.76    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# model 2

---




In [33]:
def word2features_model2(doc, i):
    word = doc[i][0]
    postag = doc[i][1]

    # Common features for all words
    features = [
        'bias',
        'word.lower=' + word.lower(),
        # 'word[-3:]=' + word[-3:],
        # 'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag
    ]


    # Features for words that are not
    # at the beginning of a document
    if i > 0:
        word1 = doc[i-1][0]
        postag1 = doc[i-1][1]
        #prev_word
        features.extend([
            'prev_word_lower='+ word1.lower(),
            'prev_word.isdigit=%s' % word1.isdigit()
        ])
    else:
        # Indicate that it is the 'beginning of a document'
        features.append('BOS')

    # Features for words that are not
    # at the end of a document
    if i < len(doc)-1:
        word1 = doc[i+1][0]
        postag1 = doc[i+1][1]

        features.extend([
            'next_word_lower='+ word1.lower(),
            'next_word.isdigit=%s' % word1.isdigit()
        ])
    else:
        # Indicate that it is the 'end of a document'
        features.append('EOS')

    return features

In [34]:
from sklearn.model_selection import train_test_split

# A function for extracting features in documents
def extract_features(doc):
    return [word2features_model2(doc, i) for i in range(len(doc))]

# A function fo generating the list of labels for each document
def get_labels(doc):
    return [label for (token, postag, label) in doc]


X = [extract_features(doc) for doc in data]
y = [get_labels(doc) for doc in data]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [35]:
import pycrfsuite
trainer = pycrfsuite.Trainer(verbose=True)

# Submit training data to the trainer
for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

# Set the parameters of the model
trainer.set_params({
    # coefficient for L1 penalty
    'c1': 0.1,

    # coefficient for L2 penalty
    'c2': 0.01,

    # maximum number of iterations
    'max_iterations': 50,

    # whether to include transitions that
    # are possible, but not observed
    'feature.possible_transitions': True
})

# Provide a file name as a parameter to the train function, such that
# the model will be saved to the file when training is finished
trainer.train('crf2.model')

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 182327
Seconds required: 3.470

L-BFGS optimization
c1: 0.100000
c2: 0.010000
num_memories: 6
max_iterations: 50
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 3591808.797122
Feature norm: 1.000000
Error norm: 3443623.558737
Active features: 181589
Line search trials: 1
Line search step: 0.000000
Seconds required for this iteration: 18.545

***** Iteration #2 *****
Loss: 2583971.646481
Feature norm: 2.522480
Error norm: 976445.454598
Active features: 178018
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 19.438

***** Iteration #3 *****
Loss: 2299433.906027
Feature norm: 2.217096
Error norm: 799669.041915
Active features: 174769
Line search trials: 1
Line search step: 1.000000
Secon

In [36]:
tagger = pycrfsuite.Tagger()
tagger.open('crf2.model')
y_pred = [tagger.tag(xseq) for xseq in X_test]

# Let's take a look at a random sample in the testing set
i = 12
for x, y in zip(y_pred[i], [x[1].split("=")[1] for x in X_test[i]]):
    print("%s (%s)" % (y, x))

'''
The following will be printed:

sci-med (N)
life (N)
systems (N)
inc (N)
said (I)
its (I)
directors (I)
approved (I)
a (I)
previously (I)
...
'''

ซึ่ง (O)
หาก (O)
ทำ (O)
ตรง (O)
นี้ (O)
ได้ (O)
ก็ (O)
เชื่อ (O)
ว่า (O)
จะ (O)
ทำให้ (O)
คุณภาพ (O)
ของ (O)
งาน (O)
รักษา (O)
ความ (O)
ปลอด (O)
ภัย (O)
เพิ่ม (O)
สูง (O)
ขึ้น (O)
นาย (B_TTL)
พงศ์ (B_PER)
กล่าว (O)


'\nThe following will be printed:\n\nsci-med (N)\nlife (N)\nsystems (N)\ninc (N)\nsaid (I)\nits (I)\ndirectors (I)\napproved (I)\na (I)\npreviously (I)\n...\n'

In [37]:
import numpy as np
from sklearn.metrics import classification_report

# Convert the sequences of tags into a 1-dimensional array
_predictions = [tag for row in y_pred for tag in row]
_truths = [tag for row in y_test for tag in row]


focus_tag = ['B_LOC', 'B_DES', 'I_MEA', 'B_ORG', 'E_ORG', 'E_BRN', 'B_BRN', 'I_TRM', 'E_TTL', 'B_TTL','B_MEA',
 'I_LOC',
 'E_DES',
 'I_BRN',
 'I_TTL',
 'I_NUM',
 'I_ORG',
 'I_PER',
 'E_TRM',
 'B_PER',
 'B_TRM',
 'E_DTM',
 'E_PER',
 'B_NUM',
 'E_LOC',
 'I_DTM',
 'E_NUM',
 'B_NAME',
 'B_DTM',
 'E_MEA',
 'I_DES']

In [38]:
truths = []
predictions = []
for x,y in zip(_truths,_predictions):
  if x in focus_tag:
    truths.append(x)
    predictions.append(y)

In [39]:
# Print out the classification report
print(classification_report(truths, predictions))

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

              precision    recall  f1-score   support

           B       0.00      0.00      0.00         0
       B_BRN       0.78      0.20      0.32       105
       B_DES       0.98      0.85      0.91      3670
       B_DTM       0.94      0.81      0.87      3174
       B_LOC       0.89      0.76      0.82      7342
       B_MEA       0.79      0.64      0.70      7485
      B_NAME       0.00      0.00      0.00         4
       B_NUM       0.57      0.59      0.58      2997
       B_ORG       0.91      0.75      0.82     10000
       B_PER       0.98      0.90      0.94      9845
       B_TRM       0.99      0.35      0.51       208
       B_TTL       1.00      0.97      0.98      4518
       E_BRN       1.00      0.12      0.21        25
       E_DES       0.93      0.72      0.81       391
       E_DTM       0.93      0.86      0.89      2677
       E_LOC       0.86      0.83      0.84      3442
       E_MEA       0.68      0.75      0.72      2137
       E_NUM       0.79    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# model 3

---



In [40]:
def word2features_model3(doc, i):
    word = doc[i][0]
    postag = doc[i][1]

    # Common features for all words
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag
    ]

    # Features for words that are not
    # at the beginning of a document
    if i > 2:
        word1 = doc[i-2][0]
        postag1 = doc[i-2][2]
        #prev_word
        features.extend([
            'prev_word_lower='+ word1.lower(),
            'prev_word.isdigit=%s' % word1.isdigit()
        ])
    else:
        # Indicate that it is the 'beginning of a document'
        features.append('BOS')

    if i > 1:
        word2 = doc[i-1][0]
        postag2 = doc[i-1][1]
        #prev_word
        features.extend([
            'prev_word_lower='+ word2.lower(),
            'prev_word.isdigit=%s' % word2.isdigit()
        ])
    else:
        # Indicate that it is the 'beginning of a document'
        features.append('BOS')


    # Features for words that are not
    # at the end of a document
    if i < len(doc)-1:
        word3 = doc[i+1][0]
        postag3 = doc[i+1][1]

        features.extend([
            'next_word_lower='+ word3.lower(),
            'next_word.isdigit=%s' % word3.isdigit()
        ])
    else:
        # Indicate that it is the 'end of a document'
        features.append('EOS')

    if i < len(doc)-2:
        word4 = doc[i+2][0]
        postag4 = doc[i+2][2]

        features.extend([
            'next_word_lower='+ word4.lower(),
            'next_word.isdigit=%s' % word4.isdigit()
        ])
    else:
        # Indicate that it is the 'end of a document'
        features.append('EOS')

    return features

In [41]:
from sklearn.model_selection import train_test_split

# A function for extracting features in documents
def extract_features(doc):
    return [word2features_model3(doc, i) for i in range(len(doc))]

# A function fo generating the list of labels for each document
def get_labels(doc):
    return [label for (token, postag, label) in doc]


X = [extract_features(doc) for doc in data]
y = [get_labels(doc) for doc in data]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [42]:
import pycrfsuite
trainer = pycrfsuite.Trainer(verbose=True)

# Submit training data to the trainer
for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

# Set the parameters of the model
trainer.set_params({
    # coefficient for L1 penalty
    'c1': 0.1,

    # coefficient for L2 penalty
    'c2': 0.01,

    # maximum number of iterations
    'max_iterations': 50,

    # whether to include transitions that
    # are possible, but not observed
    'feature.possible_transitions': True
})

# Provide a file name as a parameter to the train function, such that
# the model will be saved to the file when training is finished
trainer.train('crf3.model')

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 245709
Seconds required: 5.792

L-BFGS optimization
c1: 0.100000
c2: 0.010000
num_memories: 6
max_iterations: 50
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 2825789.627466
Feature norm: 1.000000
Error norm: 2868226.575372
Active features: 244053
Line search trials: 1
Line search step: 0.000000
Seconds required for this iteration: 23.897

***** Iteration #2 *****
Loss: 2505732.463902
Feature norm: 1.882307
Error norm: 1108642.331361
Active features: 226117
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 12.025

***** Iteration #3 *****
Loss: 2286696.309401
Feature norm: 1.666723
Error norm: 800317.264945
Active features: 238578
Line search trials: 1
Line search step: 1.000000
Seco

In [43]:
tagger = pycrfsuite.Tagger()
tagger.open('crf3.model')
y_pred = [tagger.tag(xseq) for xseq in X_test]

# Let's take a look at a random sample in the testing set
i = 12
for x, y in zip(y_pred[i], [x[1].split("=")[1] for x in X_test[i]]):
    print("%s (%s)" % (y, x))

'''
The following will be printed:

sci-med (N)
life (N)
systems (N)
inc (N)
said (I)
its (I)
directors (I)
approved (I)
a (I)
previously (I)
...
'''

โดย (O)
มี (O)
แนวโน้ม (O)
สูง (O)
ว่า (O)
_ (O)
ทั้ง (O)
นาย (B_TTL)
เนวิน (B_PER)
และ (O)
คุณหญิง (B_TTL)
สุดารัตน์ (B_PER)
จะ (O)
เดินทาง (O)
ไป (O)
สมทบ (O)
กับ (O)
_ (O)
พ.ต.ท. (B_DES)
ทักษิณ (B_PER)
ที่ (O)
กรุง (B_LOC)
ลอนดอน (E_LOC)


'\nThe following will be printed:\n\nsci-med (N)\nlife (N)\nsystems (N)\ninc (N)\nsaid (I)\nits (I)\ndirectors (I)\napproved (I)\na (I)\npreviously (I)\n...\n'

In [44]:
truths = []
predictions = []
for x,y in zip(_truths,_predictions):
  if x in focus_tag:
    truths.append(x)
    predictions.append(y)

In [45]:
# Print out the classification report
print(classification_report(truths, predictions))

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

              precision    recall  f1-score   support

           B       0.00      0.00      0.00         0
       B_BRN       0.78      0.20      0.32       105
       B_DES       0.98      0.85      0.91      3670
       B_DTM       0.94      0.81      0.87      3174
       B_LOC       0.89      0.76      0.82      7342
       B_MEA       0.79      0.64      0.70      7485
      B_NAME       0.00      0.00      0.00         4
       B_NUM       0.57      0.59      0.58      2997
       B_ORG       0.91      0.75      0.82     10000
       B_PER       0.98      0.90      0.94      9845
       B_TRM       0.99      0.35      0.51       208
       B_TTL       1.00      0.97      0.98      4518
       E_BRN       1.00      0.12      0.21        25
       E_DES       0.93      0.72      0.81       391
       E_DTM       0.93      0.86      0.89      2677
       E_LOC       0.86      0.83      0.84      3442
       E_MEA       0.68      0.75      0.72      2137
       E_NUM       0.79    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


ให้ทำการวัดผลแสดงค่า Precision Recall F1-score

**ลองทำเพิ่มเติม**

https://github.com/bentrevett/pytorch-pos-tagging

https://github.com/yoseflaw/nerindo

https://notebook.community/sameersingh/uci-statnlp/tutorials/rnn_examples

# **Model 1 : ใช้ Feature เฉพาะคำปัจจุบัน อธิบายการทำงานของ model 1**




การสร้างและฝึกอบรมโมเดล CRF สำหรับการทำนาย Named Entities (NE) ในประโยคภาษาไทย โดยมี feature ที่ถูกสร้างโดยฟังก์ชัน word2features ซึ่งถูกนำไปใช้ในการสร้าง feature sequences (X_train) และ label sequences (y_train) สำหรับการฝึกอบรมโมเดล CRF ดังนี้:

1.Feature Extraction (word2features function):

word.lower: ค่า lowercase ของคำปัจจุบัน
word.isupper: บอกว่าคำปัจจุบันมีตัวอักษรทุกตัวเป็น uppercase หรือไม่
word.istitle: บอกว่าคำปัจจุบันมีการใช้ตัวใหญ่ตัวแรกในคำหรือไม่ (title case)
word.isdigit: บอกว่าคำปัจจุบันมีตัวเลขหรือไม่
postag: ประเภทของคำปัจจุบัน (part-of-speech tag)
BOS (beginning of a document): บอกว่าเป็นคำแรกของเอกสารหรือไม่
EOS (end of a document): บอกว่าเป็นคำสุดท้ายของเอกสารหรือไม่

2.Training the CRF Model (pycrfsuite.Trainer):

โค้ดสร้างตัวแบบ CRF โดยใช้ pycrfsuite.Trainer และนำข้อมูลที่ได้จาก word2features ไปฝึกอบรม
ตั้งค่าพารามิเตอร์ต่าง ๆ เช่น coefficient สำหรับ L1 และ L2 penalty, จำนวนการฝึกอบรม (max_iterations), และการรวม transition ที่เป็นไปได้ (feature.possible_transitions)

3.Testing the CRF Model (pycrfsuite.Tagger):

โค้ดนำโมเดลที่ได้จากการฝึกอบรมมาใช้ในการทำนาย NE สำหรับชุดข้อมูลทดสอบ (X_test)
แสดงผลลัพธ์การทำนายของโมเดลสำหรับตัวอย่างข้อมูลในชุดทดสอบ
Evaluation (classification_report):

โค้ดนำผลลัพธ์การทำนายของโมเดลมาวัดประสิทธิภาพโดยใช้ classification_report และให้ค่า precision, recall, f1-score สำหรับแต่ละกลุ่ม NE ที่กำหนดใน focus_tag
โมเดล CRF นี้ถูกออกแบบให้สามารถทำนาย NE ในประโยคภาษาไทยได้ โดยใช้ลักษณะทางด้านภาษาและบริบทของคำที่ถูกกำหนดโดย word2features และมีการประเมินประสิทธิภาพด้วย classification_report.

# **Model 2 : ใช้ Feature ของคำปัจจุบัน คำก่อนหน้าและคำถัดไป (w_i-1,w_i,w_i+1) อธิบายการทำงานของ model 2**

การสร้างและฝึกอบรมโมเดล CRF สำหรับการทำนาย Named Entities (NE) ในประโยคภาษาไทย โดย Model 2 ใช้ Feature ของคำปัจจุบัน (w_i), คำก่อนหน้า (w_i-1), และคำถัดไป (w_i+1) ในฟังก์ชัน word2features_model2.

1.Feature Extraction (word2features_model2 function):
word.lower: ค่า lowercase ของคำปัจจุบัน
word.isupper: บอกว่าคำปัจจุบันมีตัวอักษรทุกตัวเป็น uppercase หรือไม่
word.istitle: บอกว่าคำปัจจุบันมีการใช้ตัวใหญ่ตัวแรกในคำหรือไม่ (title case)
word.isdigit: บอกว่าคำปัจจุบันมีตัวเลขหรือไม่
postag: ประเภทของคำปัจจุบัน (part-of-speech tag)
prev_word_lower: lowercase ของคำก่อนหน้า
prev_word.isdigit: บอกว่าคำก่อนหน้ามีตัวเลขหรือไม่
next_word_lower: lowercase ของคำถัดไป
next_word.isdigit: บอกว่าคำถัดไปมีตัวเลขหรือไม่
BOS (beginning of a document): บอกว่าเป็นคำแรกของเอกสารหรือไม่
EOS (end of a document): บอกว่าเป็นคำสุดท้ายของเอกสารหรือไม่

2.Training the CRF Model (pycrfsuite.Trainer):
โค้ดสร้างตัวแบบ CRF โดยใช้ pycrfsuite.Trainer และนำข้อมูลที่ได้จาก word2features_model2 ไปฝึกอบรม
ตั้งค่าพารามิเตอร์ต่าง ๆ เช่น coefficient สำหรับ L1 และ L2 penalty, จำนวนการฝึกอบรม (max_iterations), และการรวม transition ที่เป็นไปได้ (feature.possible_transitions)

3.Testing the CRF Model (pycrfsuite.Tagger):
โค้ดนำโมเดลที่ได้จากการฝึกอบรมมาใช้ในการทำนาย NE สำหรับชุดข้อมูลทดสอบ (X_test)
แสดงผลลัพธ์การทำนายของโมเดลสำหรับตัวอย่างข้อมูลในชุดทดสอบ

4.Evaluation (classification_report):
โค้ดนำผลลัพธ์การทำนายของโมเดลมาวัดประสิทธิภาพโดยใช้ classification_report และให้ค่า precision, recall, f1-score สำหรับแต่ละกลุ่ม NE ที่กำหนดใน focus_tag
โมเดล CRF นี้ถูกออกแบบให้สามารถทำนาย NE ในประโยคภาษาไทยได้ โดยใช้ลักษณะทางด้านภาษาและบริบทของคำเพื่อประมวลผล

# **Model 3 : ใช้ Fature เหมือนโมเดล 2 แต่เพิ่มคำรอบข้างอีก 1 คำ (w_i-2,w_i-1,w_i,w_i+1,w_i+2) อธิบาย Model 3**

สร้างและฝึกอบรมโมเดล CRF สำหรับทำนาย Named Entities (NE) ในประโยคภาษาไทยโดย Model 3 ใช้ Feature ของคำปัจจุบัน (w_i), คำก่อนหน้าและคำถัดไป (w_i-1, w_i+1), และคำรอบข้าง (w_i-2, w_i-1, w_i+1, w_i+2) ในฟังก์ชัน word2features_model3.

1.Feature Extraction (word2features_model3 function):
word.lower: ค่า lowercase ของคำปัจจุบัน
word.isupper: บอกว่าคำปัจจุบันมีตัวอักษรทุกตัวเป็น uppercase หรือไม่
word.istitle: บอกว่าคำปัจจุบันมีการใช้ตัวใหญ่ตัวแรกในคำหรือไม่ (title case)
word.isdigit: บอกว่าคำปัจจุบันมีตัวเลขหรือไม่
postag: ประเภทของคำปัจจุบัน (part-of-speech tag)
prev_word_lower: lowercase ของคำก่อนหน้า (2 คำก่อนหน้า)
prev_word.isdigit: บอกว่าคำก่อนหน้ามีตัวเลขหรือไม่ (2 คำก่อนหน้า)
next_word_lower: lowercase ของคำถัดไป (2 คำถัดไป)
next_word.isdigit: บอกว่าคำถัดไปมีตัวเลขหรือไม่ (2 คำถัดไป)
BOS (beginning of a document): บอกว่าเป็นคำแรกของเอกสารหรือไม่
EOS (end of a document): บอกว่าเป็นคำสุดท้ายของเอกสารหรือไม่

2.Training the CRF Model (pycrfsuite.Trainer):
โค้ดสร้างตัวแบบ CRF โดยใช้ pycrfsuite.Trainer และนำข้อมูลที่ได้จาก word2features_model3 ไปฝึกอบรม
ตั้งค่าพารามิเตอร์ต่าง ๆ เช่น coefficient สำหรับ L1 และ L2 penalty, จำนวนการฝึกอบรม (max_iterations), และการรวม transition ที่เป็นไปได้ (feature.possible_transitions)

3.Testing the CRF Model (pycrfsuite.Tagger):
โค้ดนำโมเดลที่ได้จากการฝึกอบรมมาใช้ในการทำนาย NE สำหรับชุดข้อมูลทดสอบ (X_test)
แสดงผลลัพธ์การทำนายของโมเดลสำหรับตัวอย่างข้อมูลในชุดทดสอบ
Evaluation (classification_report):

โค้ดนำผลลัพธ์การทำนายของโมเดลมาวัดประสิทธิภาพโดยใช้ classification_report และให้ค่า precision, recall, f1-score สำหรับแต่ละกลุ่ม NE ที่กำหนดใน focus_tag